In [11]:
import os, posixpath
import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from azure.storage.blob import BlobServiceClient
import dlt

PROFILE   = "s3_to_adls"  # el nombre que usaste en secrets.toml

src_cfg   = dlt.secrets[f"{PROFILE}.sources.filesystem"]
src_creds = dlt.secrets[f"{PROFILE}.sources.filesystem.credentials"]

# --- MinIO (ya montado por compose) ---
# Arriba se definen variables para llamar mas sencillo a Credenciales o Configuraciones, asi guiarnos con secrets.
MINIO_ENDPOINT   = src_creds["endpoint_url"]
MINIO_ACCESS_KEY = src_creds["aws_access_key_id"]
MINIO_SECRET_KEY = src_creds["aws_secret_access_key"]
MINIO_BUCKET     = src_cfg["bucket_url"].replace("s3://", "").split("/", 1)[0]

SOURCE_KEY = "yellow_trip_20250910112135/yellow_trip_data/1757546495.2376165.503b80e90a.parquet"

s3 = boto3.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY,
    config=Config(signature_version="s3v4"),
    region_name=os.getenv("AWS_REGION","us-east-1"),
)

# Verifica que existe en MinIO, esto para que así nos acordemos de ejecutar el primer notebook!!!
try:
    head = s3.head_object(Bucket=MINIO_BUCKET, Key=SOURCE_KEY)
    size = head.get("ContentLength", 0)
    print(f"✓ Encontrado en MinIO: s3://{MINIO_BUCKET}/{SOURCE_KEY} ({size/1024:.1f} KiB)")
except ClientError as e:
    raise RuntimeError(f"No se encontró el objeto en MinIO: s3://{MINIO_BUCKET}/{SOURCE_KEY}") from e

✓ Encontrado en MinIO: s3://warehouse/yellow_trip_20250910112135/yellow_trip_data/1757546495.2376165.503b80e90a.parquet (70411.2 KiB)


In [4]:
import dlt

# Asi usar las credenciales desde secrets.toml
creds = dlt.secrets["s3_to_adls.destination.filesystem.credentials"]
account_name = creds["azure_storage_account_name"]
account_key  = creds["azure_storage_account_key"]

from azure.storage.blob import BlobServiceClient
blob_service = BlobServiceClient(
    account_url=f"https://{account_name}.blob.core.windows.net",
    credential=account_key
)

# Info del contenedor
dest_info = dlt.secrets["s3_to_adls.destination.filesystem"]
container_name = dest_info["bucket_url"].split("@")[0].replace("abfss://","")

container = blob_service.get_container_client(container_name)
try:
    container.create_container()
except Exception:
    pass

print("Cliente Azure desde secrets.toml listo ✅")

Cliente Azure desde secrets.toml listo ✅


In [8]:
SOURCE_KEY = "yellow_trip_20250910112135/yellow_trip_data/1757546495.2376165.503b80e90a.parquet"
dest_key   = f"GRUPO_1/{SOURCE_KEY}"

obj = s3.get_object(Bucket=MINIO_BUCKET, Key=SOURCE_KEY)
container.upload_blob(name=dest_key, data=obj["Body"], overwrite=True)

print(f"✅ Copiado a Azure (via secrets): {container_name}/{dest_key}")


✅ Copiado a Azure (via secrets): clase-4-dlt/GRUPO_1/yellow_trip_20250910112135/yellow_trip_data/1757546495.2376165.503b80e90a.parquet
